In [11]:
import pandas as pd

In [12]:
df_train = pd.read_csv("07_data/train_data.csv")
df_test = pd.read_csv("07_data/test_data.csv")

In [13]:
df_train.head()

,ID,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,...,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,1,27,79,Low,High,Yes,8,63,High,Yes,...,Low,Medium,Public,Negative,5,No,College,Moderate,Female,69
1,2,16,86,High,Medium,Yes,7,94,Medium,Yes,...,Low,High,Public,Neutral,3,No,High School,Moderate,Female,69
2,3,22,87,Low,Medium,No,8,83,Low,Yes,...,Low,Medium,Public,Neutral,1,No,College,Far,Male,66
3,4,18,100,High,Medium,Yes,10,86,Medium,Yes,...,Medium,Medium,Public,Neutral,3,No,High School,Near,Male,72
4,5,35,78,High,Low,Yes,10,99,Medium,Yes,...,Low,Medium,Private,Positive,2,No,High School,Near,Male,72


In [14]:
df_train.info()
df_train.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5285 entries, 0 to 5284
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          5285 non-null   int64 
 1   Hours_Studied               5285 non-null   int64 
 2   Attendance                  5285 non-null   int64 
 3   Parental_Involvement        5285 non-null   object
 4   Access_to_Resources         5285 non-null   object
 5   Extracurricular_Activities  5285 non-null   object
 6   Sleep_Hours                 5285 non-null   int64 
 7   Previous_Scores             5285 non-null   int64 
 8   Motivation_Level            5285 non-null   object
 9   Internet_Access             5285 non-null   object
 10  Tutoring_Sessions           5285 non-null   int64 
 11  Family_Income               5285 non-null   object
 12  Teacher_Quality             5225 non-null   object
 13  School_Type                 5285 non-null   obje

,ID,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,...,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
count,5285.000000,5285.000000,5285.000000,5285,5285,5285,5285.000000,5285.000000,5285,5285,...,5285,5225,5285,5285,5285.000000,5285,5216,5234,5285,5285.000000
unique,NaN,NaN,NaN,3,3,2,NaN,NaN,3,2,...,3,3,2,3,NaN,2,3,3,2,NaN
top,NaN,NaN,NaN,Medium,Medium,Yes,NaN,NaN,Medium,Yes,...,Low,Medium,Public,Positive,NaN,No,High School,Near,Male,NaN
freq,NaN,NaN,NaN,2690,2663,3163,NaN,NaN,2688,4873,...,2146,3141,3683,2152,NaN,4704,2591,3090,3054,NaN
mean,2643.000000,19.967077,79.873605,NaN,NaN,NaN,7.024409,75.061116,NaN,NaN,...,NaN,NaN,NaN,NaN,2.968590,NaN,NaN,NaN,NaN,67.215137
std,1525.792417,5.993282,11.519161,NaN,NaN,NaN,1.458215,14.406537,NaN,NaN,...,NaN,NaN,NaN,NaN,1.033942,NaN,NaN,NaN,NaN,3.922228
min,1.000000,1.000000,60.000000,NaN,NaN,NaN,4.000000,50.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,56.000000
25%,1322.000000,16.000000,70.000000,NaN,NaN,NaN,6.000000,63.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,65.000000
50%,2643.000000,20.000000,80.000000,NaN,NaN,NaN,7.000000,75.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,67.000000
75%,3964.000000,24.000000,90.000000,NaN,NaN,NaN,8.000000,88.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,69.000000


In [15]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


def add_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["study_efficiency"] = out["Hours_Studied"] / (out["Tutoring_Sessions"] + 1)
    out["attendance_ratio"] = out["Attendance"] / 100.0
    out["sleep_quality"] = out["Sleep_Hours"] * out["attendance_ratio"]
    out["resource_and_parents"] = (
        out["Parental_Involvement"].astype(str)
        + "_"
        + out["Access_to_Resources"].astype(str)
    )

    out["Hours_Studied_sq"] = out["Hours_Studied"] ** 2
    out["Previous_Scores_sq"] = out["Previous_Scores"] ** 2

    return out


df_train_fe = add_features(df_train)
df_test_fe = add_features(df_test)

In [16]:
TARGET = "Exam_Score"
ID_COL = "ID"
feature_cols = [c for c in df_train_fe.columns if c not in [TARGET, ID_COL]]

# separăm numerical vs categorical
cat_cols = df_train_fe[feature_cols].select_dtypes("object").columns.tolist()
num_cols = [c for c in feature_cols if c not in cat_cols]

numeric_trf = Pipeline(
    [
        ("imp", SimpleImputer(strategy="median")),
        ("sc", StandardScaler()),
    ]
)
cat_trf = Pipeline(
    [
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh", OneHotEncoder(handle_unknown="ignore")),
    ]
)
pre = ColumnTransformer(
    [
        ("num", numeric_trf, num_cols),
        ("cat", cat_trf, cat_cols),
    ]
)

model = GradientBoostingRegressor(random_state=42, loss="huber", alpha=0.9)
pipe = Pipeline([("pre", pre), ("gb", model)])

In [17]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = -cross_val_score(
    pipe,
    df_train_fe[feature_cols],
    df_train_fe[TARGET],
    cv=cv,
    scoring="neg_mean_absolute_error",
)
print(f"MAE (5‑fold): mean={mae_scores.mean():.3f} ± {mae_scores.std():.3f}")

MAE (5‑fold): mean=0.808 ± 0.046


In [18]:
pipe.fit(df_train_fe[feature_cols], df_train_fe[TARGET])
df_test["pred_exam_score"] = pipe.predict(df_test_fe[feature_cols])

In [19]:
mean_hours = df_train["Hours_Studied"].mean()

# Subtask 1
df_test["diff_hours_abs"] = (df_test["Hours_Studied"] - mean_hours).abs().round(2)

# Subtask 2
df_test["sleep_few"] = df_test["Sleep_Hours"] < 7

# Subtask 3
prev_scores_train = df_train["Previous_Scores"]
df_test["count_prev_ge"] = df_test["Previous_Scores"].apply(
    lambda v: int((prev_scores_train >= v).sum())
)

# Subtask 4
mot_counts = df_train["Motivation_Level"].value_counts()
df_test["count_same_motivation"] = (
    df_test["Motivation_Level"].map(mot_counts).fillna(0).astype(int)
)

In [20]:
rows = []
for _, row in df_test.iterrows():
    idx = int(row[ID_COL])
    rows.extend(
        [
            (1, idx, float(row["diff_hours_abs"])),
            (2, idx, bool(row["sleep_few"])),
            (3, idx, int(row["count_prev_ge"])),
            (4, idx, int(row["count_same_motivation"])),
            (5, idx, int(round(row["pred_exam_score"], 2))),
        ]
    )

submission = pd.DataFrame(rows, columns=["subtaskID", "datapointID", "answer"])
submission.to_csv("07_data/submission.csv", index=False)